<a href="https://colab.research.google.com/github/talhaanwarch/Profiling-Hate-Speech-Spreaders-on-Twitter/blob/main/Notebooks/Sentence/English/autokeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f10d5356-35a1-3fe4-19b1-3dfe76a2243b)


#install pacakges

In [2]:
%%capture
!python3 -m pip install autokeras

# download data

In [3]:
import os
if os.path.isdir('/content/Profiling-Hate-Speech-Spreaders-on-Twitter') is False:
    !git clone https://github.com/talhaanwarch/Profiling-Hate-Speech-Spreaders-on-Twitter.git

Cloning into 'Profiling-Hate-Speech-Spreaders-on-Twitter'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 80 (delta 22), reused 31 (delta 7), pack-reused 0
Unpacking objects: 100% (80/80), done.


In [4]:
with open('pass.txt','r') as f:
  pas=f.read()

In [5]:
import zipfile
if os.path.isdir('train') is False:
  train_path='/content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/train.zip'
  with zipfile.ZipFile(train_path,"r") as zip_ref:
    zip_ref.extractall(".",pwd = bytes(pas,'utf-8'))
if os.path.isdir('test') is False:
  test_path='/content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/test.zip'
  with zipfile.ZipFile(test_path,"r") as zip_ref:
    zip_ref.extractall(".",pwd = bytes(pas,'utf-8'))

# prepare data

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df=pd.read_csv('train/en/truth.txt',sep=':::',header=None,engine='python')
df.columns=['id','label']

In [7]:
df.label.value_counts()

1    100
0    100
Name: label, dtype: int64

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
def preprocessing(text):
    text=text.str.replace('\d+', '')
    text=text.str.replace('RT','')
    text=text.str.replace('#USER#','')
    text=text.str.replace('#URL#','')
    text= text.str.lower()
    text = text.str.replace('[^\w\s]','')
    #text = text.apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
    #text = text.apply(lambda x: [item for item in x if item not in stop])
    #text = text.apply(lambda x : " ".join(x))
    return text

In [10]:
df.id='train/en/'+df.id.astype(str)+'.xml'
df.head()

,id,label
0,train/en/639b8e5e6a527d494c85d8f5704b1a01.xml,0
1,train/en/f2b1fc84c500c38a93522efbd422b559.xml,0
2,train/en/10b2d013382e1fb3c9414ea28329f258.xml,0
3,train/en/26644d1348fc1122e8c5ef45d6bc84fa.xml,0
4,train/en/4d4c5dcbfe38d0d33a0d1b1419952ca8.xml,0


In [11]:
len(df)

200

In [12]:
import xml.etree.ElementTree as ET
def reader(df,ground=True):
  data=[]
  for x in df.iterrows():
      
      tree = ET.parse(x[1].id)
      root = tree.getroot()
      text=[x.text for x in root[0]]
      if ground:
        label=[x[1].label]*len(text)
        data.append(pd.DataFrame(zip(text,label),columns=['text','label']))
      else:
        data.append(pd.DataFrame(text,columns=['text']))
  return data

In [13]:
data=reader(df)

In [14]:
data[0].head()

,text,label
0,RT #USER#: Funny how “15 days to slow the spre...,0
1,RT #USER#: Why did Minneapolis just give Georg...,0
2,"RT #USER#: To be fair, he has done a lot of un...",0
3,RT #USER#: President Trump got us the #HASHTAG...,0
4,RT #USER#: Is the case against former officer ...,0


In [16]:
len(data),len(data[0])

(200, 200)

In [17]:
from sklearn.model_selection import train_test_split
trainx,valx=train_test_split(data,test_size=0.1,)
trainx,testx=train_test_split(trainx,test_size=0.1)

In [18]:
train=pd.concat(trainx)
val=pd.concat(valx)
test=pd.concat(testx)

In [19]:
train.label.value_counts()

0    16600
1    15800
Name: label, dtype: int64

In [20]:
val.label.value_counts()

1    2200
0    1800
Name: label, dtype: int64

In [21]:
test.label.value_counts()

1    2000
0    1600
Name: label, dtype: int64

In [22]:
train.text=preprocessing(train.text)
val.text=preprocessing(val.text)
test.text=preprocessing(test.text)

In [23]:
len(train),len(val),len(test)

(32400, 4000, 3600)

In [24]:
train = train.sample(frac=1).reset_index(drop=True)
val = val.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)


In [25]:
train_text=train.text.values
train_label=train.label.values
val_text=val.text.values
val_label=val.label.values
test_text=test.text.values
test_label=test.label.values

# classify data

In [26]:
import autokeras as ak
clf = ak.TextClassifier() 

In [ ]:
clf.fit(train_text,train_label,epochs=100,validation_data=(val_text, val_label))


Trial 2 Complete [00h 03m 40s]
val_loss: 0.7010567784309387

Best val_loss So Far: 0.7010567784309387
Total elapsed time: 00h 04m 18s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
text_block_1/bl...|transformer       |vanilla           
classification_...|0                 |0                 
optimizer         |adam              |adam              
learning_rate     |0.001             |0.001             
text_block_1/ma...|20000             |5000              
text_block_1/te...|200               |512               
text_block_1/tr...|none              |None              
text_block_1/tr...|32                |None              
text_block_1/tr...|2                 |None              
text_block_1/tr...|32                |None              
text_block_1/tr...|0.25              |None              
text_block_1/sp...|global_avg        |global_max        
text_block_1/de...|1                 |1                 
text_block_1/de...|False             |Fal

In [ ]:
val_score = clf.evaluate(val_text, val_label)

In [ ]:
test_score = clf.evaluate(test_text,test_label)

In [ ]:
from sklearn.metrics import classification_report
pred=clf.predict(val_text)
print(classification_report(val_label,pred))

In [ ]:
pred=clf.predict(test_text)
print(classification_report(test_label,pred))

# validation

In [ ]:
preds,trues=[],[]
for i in valx:
  pred=list(clf.predict(i.text.values,verbose=0))
  preds.append(max(pred,key=pred.count))
  trues.append(int(np.mean(i.label)))
print(classification_report(trues,preds))  

In [ ]:
preds,trues=[],[]
for i in testx:
  pred=list(clf.predict(i.text.values,verbose=0))
  preds.append(max(pred,key=pred.count))
  trues.append(int(np.mean(i.label)))
print(classification_report(trues,preds))  

# test evaluation

In [ ]:
from glob import glob
test_path=pd.DataFrame(glob('test/en/*.xml'),columns=['id'])
test_path.head()

,id
0,test/en/f2973063f16c0698a6de86c55b5f9ad6.xml
1,test/en/be71e41921653e523aceb555727b61c7.xml
2,test/en/d0235c3e74f79298fc2b6d2427c2060b.xml
3,test/en/314d4bf068314367bc83dc5ae3757e13.xml
4,test/en/fdd27881da241cff7115c1e89caba6dd.xml


In [ ]:
test=reader(test_path,ground=False)

In [ ]:
test[0].head()

,text
0,#USER# I'm so sorry 😞...
1,#USER# #USER# That is correct.
2,RT #USER#: The legitimate President of the Uni...
3,#USER# #USER# #USER# Texas AG is suing the Bid...
4,#USER# In Jesus Holy Name. Soon the entire wor...


In [ ]:
final_preds=[]
for i in range(len(test)):
  test[i].text=preprocessing(test[i].text)
  pred=list(clf.predict(test[i].text.values,verbose=0))
  final_preds.append(int(max(pred,key=pred.count)))


In [ ]:
len(final_preds),len(test_path)

In [ ]:
ids=test_path.id.apply(lambda x:x.split('/')[-1].split('.')[0])

In [ ]:
lang=['en']*len(test_path)

In [ ]:
final=pd.DataFrame(zip(ids,lang,final_preds),columns=['author-id','lang','type'])
final.head()